In [79]:
import sys
sys.path.append('../../')
import os

import pandas as pd
from utils import fill_empty, VColumns
from dotenv import load_dotenv
load_dotenv()

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load input file

In [80]:
df1 = pd.read_csv("v1-start.csv")

# Start elaborating columns

Create a new dataframe with empty values, so that we can start filling in values from the input dataframe, without overriding columns.

In [81]:
# create new empty dataframe
df = pd.DataFrame(columns=VColumns.v2())
df.head()

,external_id,type,name,winery_name,info,size,vintage,price,purchase_price,quantity,storage_area,internal_notes,visible


In [82]:
# copy values from original columns to new columns
df['external_id'] = df1['external_id']
df['type'] = df1['type']
df['name'] = df1['name'].astype(str) + df1['style'].fillna('').astype(str) # concatenate name and style
df['winery_name'] = df1['winery_name']

# Extract 4-digit vintage (e.g. 2019) from name by:
# 1. Getting all digits from name string
# 2. Taking last 4 digits if there are at least 4 digits
# 3. Converting to int if valid vintage found, otherwise None
df["vintage"] = df1.apply(lambda x: int(''.join(c for c in str(x["name"]) if c.isdigit())[-4:]) if pd.notna(x["name"]) and any(c.isdigit() for c in str(x["name"])) and len(''.join(c for c in str(x["name"]) if c.isdigit())) >= 4 else None, axis=1)
df["quantity"] = df1["qty"]
df["storage_area"] = None
df["info"] = df1["info"]
df["internal_notes"] = None
df["visible"] = True

df.head()

,external_id,type,name,winery_name,info,size,vintage,price,purchase_price,quantity,storage_area,internal_notes,visible
0,300001.0,NaN,rose NV,Bollinger,NaN,NaN,NaN,NaN,NaN,1.0,None,None,True
1,300002.0,NaN,"Grande Cuvée (PN, CH, PM) NV",Krug,NaN,NaN,NaN,NaN,NaN,2.0,None,None,True
2,NaN,NaN,~,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,True
3,300500.0,NaN,Special Cuvee NV,Bollinger,NaN,NaN,NaN,NaN,NaN,24.0,None,None,True
4,300501.0,NaN,Special Cuvee NV,Bollinger,NaN,NaN,NaN,NaN,NaN,18.0,None,None,True


In [83]:
print("Before removing empty lines: ", len(df))

# Drop empty lines
df = df[df['name'].fillna('').str.len() + df['winery_name'].fillna('').str.len() > 5]

print("After removing empty lines: ", len(df))

Before removing empty lines:  3411
After removing empty lines:  3071


In [84]:
# Map sizes 
print(df1['size'].unique())

map_sizes = {
    0.375: 'HALF_BOTTLE',
    0.75: 'BOTTLE',
    0.72: 'BOTTLE',
    0.7: 'BOTTLE',
    1.5: 'MAGNUM',
    3: 'JEROBOAM',
    5: 'BORDEAUX_JEROBOAM',
}

df['size'] = df1['size'].apply(lambda x: map_sizes[x] if x in map_sizes else 'BOTTLE')

print(df['size'].unique())


[0.375   nan 0.75  1.5   3.    5.    0.72  0.5   0.7  ]
['HALF_BOTTLE' 'BOTTLE' 'MAGNUM' 'JEROBOAM' 'BORDEAUX_JEROBOAM']


In [86]:
def convert_price_to_cents(x):
    if pd.isna(x) or str(x).strip() == '':
        return 0
    try:
        return int(float(str(x).replace('€', '').strip()) * 100)
    except (ValueError, TypeError):
        return 0

df1['sales_price_eur'] = df1['sales_price_eur'].fillna('')
df['price'] = df1['sales_price_eur'].apply(convert_price_to_cents)

df1['purchase_price_eur'] = df1['purchase_price_eur'].fillna('')
df['purchase_price'] = df1['purchase_price_eur'].apply(convert_price_to_cents)

ValueError: could not convert string to float: ''

In [38]:
df = fill_empty(df, VColumns.v2(), False)
df.head()

,external_id,type,name,winery_name,info,size,vintage,price,purchase_price,quantity,storage_area,internal_notes,visible,purchase_price_inclvat
0,,RED,Chianti Classico,Agricoltori del Chianti Geografico,,BOTTLE,1971,0,81,1,Collezione,,True,1 €
1,,RED,Amarone della Valpolicella Classico docg,Allegrini,,BOTTLE,2017,15000,6229,4,s2 - legno,,True,76 €
2,,RED,Amarone della Valpolicella Classico docg,Allegrini,,BOTTLE,2019,12000,4344,5,s2 - legno,,True,53 €
3,,RED,La Grola igt Rosso Veronese,Allegrini,,BOTTLE,2018,0,0,0,nan,,True,0
4,,RED,La Grola igt Rosso Veronese,Allegrini,,BOTTLE,2019,4500,1475,17,s4+bar005,,True,18 €


# Merge files and write output

If there are multiple files, merge them into one and create single output file.

In [39]:
df.head()

df_out = pd.concat([df], ignore_index=True)
df_out.to_csv("v2-cleaned.csv", index=False)

bool()

False